In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [2]:
model = AutoModelForTokenClassification.from_pretrained("yeye776/koelectra-finetuned-ner")

config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

2024-04-08 09:03:14.274471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


pytorch_model.bin:   0%|          | 0.00/56.3M [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("yeye776/koelectra-finetuned-ner")

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/815k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [4]:
sentence="유선상으로 말씀하신 거래대금 120만원은 국민은행 123-4567-890 예금주 김철수로 12월 15일 오후 3시 전까지 송금해주세요. \
혹시 문제가 있다면 asdfgh@gmail.com으로 메일 보내주세요. 제 주소는 서울시 성북구 보문로 12길 345호입니다."

In [5]:
tokens = tokenizer([sentence], return_tensors="pt")

In [6]:
outputs = model(**tokens)

In [7]:
predictions = outputs['logits'].argmax(dim=-1)

In [9]:
mask2word={}
mask_list=[]
word_list=[]
is_entity=False
prev=None

In [10]:
for n, (t,o) in enumerate(zip(tokens['input_ids'][0].tolist()[1:-1], predictions[0].tolist()[1:-1])):
    word = tokenizer.decode(t)
    label = model.config.id2label[o][-2:]
    if label == prev:
        prev_word, prev_label = word_list.pop()
        if word[:2] == "##":
            word_list.append((prev_word+word[2:], prev_label))
        else:
            word_list.append((prev_word, prev_label))
            word_list.append((word, label))
    else:
        if word[:2] == "##":
            word_list.append((word[2:], label))
        else:
            word_list.append((word, label))
    prev = label
print (word_list)

[('유선상으로', 'O'), ('말씀하신', 'O'), ('거래대금', 'O'), ('120만원', 'QT'), ('은', 'O'), ('국민은행', 'OG'), ('123', 'QT'), ('-', 'QT'), ('4567', 'QT'), ('-', 'QT'), ('890', 'O'), ('예금주', 'O'), ('김철수', 'PS'), ('로', 'O'), ('12월', 'DT'), ('15일', 'DT'), ('오후', 'TI'), ('3시', 'TI'), ('전까지', 'TI'), ('송금해주세요', 'O'), ('.', 'O'), ('혹시', 'O'), ('문제가', 'O'), ('있다면', 'O'), ('asdfgh', 'TM'), ('@', 'TM'), ('gmail', 'TM'), ('.', 'TM'), ('com', 'TM'), ('으로', 'O'), ('메일', 'O'), ('보내주세요', 'O'), ('.', 'O'), ('제', 'O'), ('주소는', 'O'), ('서울시', 'LC'), ('성북구', 'LC'), ('보문로', 'LC'), ('12길', 'O'), ('345호입니다', 'O'), ('.', 'O')]


In [11]:
# O : 일반단어
# QT : 수
# OG : 기관
# PS : 사람
# DT : 날짜
# TI : 시간

In [14]:
count = 0
masked_sentence = sentence
mask2word = {}
for word, tag in word_list:
    if tag != "O" and tag != "QT":
        mask_token = f"###{tag}-{count}###"
        masked_sentence = masked_sentence.replace(word, mask_token)
        mask2word[mask_token] = word
        count += 1
print(masked_sentence)

유선상으로 말씀하신 거래대금 120만원은 ###OG-0### 123-4567-890 예금주 ###PS-1###로 ###DT-2### ###DT-3### ###TI-4### ###TI-5### ###TI-6### 송금해주세요###TM-10### 혹시 문제가 있다면 ###TM-7######TM-8######TM-9######TM-10######TM-11###으로 메일 보내주세요###TM-10### 제 주소는 ###LC-12### ###LC-13### ###LC-14### 12길 345호입니다###TM-10###


In [17]:
for k in mask2word:
    masked_sentence = masked_sentence.replace(k, mask2word[k])
print(masked_sentence)

유선상으로 말씀하신 거래대금 120만원은 국민은행 123-4567-890 예금주 김철수로 12월 15일 오후 3시 전까지 송금해주세요. 혹시 문제가 있다면 asdfgh@gmail.com으로 메일 보내주세요. 제 주소는 서울시 성북구 보문로 12길 345호입니다.


In [16]:
mask2word

{'###OG-0###': '국민은행',
 '###PS-1###': '김철수',
 '###DT-2###': '12월',
 '###DT-3###': '15일',
 '###TI-4###': '오후',
 '###TI-5###': '3시',
 '###TI-6###': '전까지',
 '###TM-7###': 'asdfgh',
 '###TM-8###': '@',
 '###TM-9###': 'gmail',
 '###TM-10###': '.',
 '###TM-11###': 'com',
 '###LC-12###': '서울시',
 '###LC-13###': '성북구',
 '###LC-14###': '보문로'}

In [18]:
!pip install Faker

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.2 MB/s eta 0:00:00


In [32]:
from faker import Faker
fake = Faker('ko_KR')
print(fake.name())
print(fake.address())
print(fake.time())
print(fake.date())
print(fake.phone_number())
print(fake.ssn())
print(fake.pricetag())
print(fake.profile())

이은경
서울특별시 동작구 석촌호수084거리
22:47:43
1985-10-10
017-153-2461
870620-2951153
$0.81
{'job': '노점 및 이동 판매원', 'company': '황최김', 'ssn': '830701-2891381', 'residence': '인천광역시 마포구 가락2가 (옥순이마을)', 'current_location': (Decimal('16.435941'), Decimal('-134.719483')), 'blood_group': 'O-', 'website': ['https://simi.kr/', 'http://www.ju.com/'], 'username': 'jia33', 'name': '유민지', 'sex': 'F', 'address': '대전광역시 동작구 서초대1거리', 'mail': 'bagyeongho@gmail.com', 'birthdate': datetime.date(1984, 3, 24)}


In [29]:
import random

mask2word = {}
count = 0
masked_sentence = []

entity_list = []
prev = None
for word, tag in word_list:
    if tag == prev:
        prev_word, prev_label = entity_list.pop()
        entity_list.append((prev_word+' '+word, prev_label))
    else:
        entity_list.append((word, tag))
    prev = tag

for word, tag in entity_list:
    if tag == "O":
        masked_sentence.append(word)
    else:
        if tag == "QT":
            if word[-1] == "원":
                mask_token = fake.pricetag()
            else:
                mask_token = "".join([str(random.randint(0,9)) for _ in word])
        elif tag == "OG":
            if word[-2:] == "은행":
                mask_token = fake.company() + "은행"
        elif tag == "PS":
            mask_token = fake.name()
        elif tag == "DT":
            mask_token = fake.date()
        elif tag == "TI":
            mask_token = fake.time()
        elif tag == "TM":
            if "@" in word:
                mask_token = fake.profile()['mail']
        elif tag[:3] == "LC":
            mask_token = fake.city()

        mask2word[mask_token] = word
        masked_sentence.append(mask_token)
        count +=1


In [30]:
print(masked_sentence)

['유선상으로 말씀하신 거래대금', '$698.45', '은', '정박은행', '042017062752', '890 예금주', '안상훈', '로', '2017-07-01', '13:16:14', '송금해주세요 . 혹시 문제가 있다면', 'yeeun28@live.com', '으로 메일 보내주세요 . 제 주소는', '연천군', '12길 345호입니다 .']
